In [1]:
import sys
sys.path.append('/home/xinyiz/pamrats')

In [2]:
import time
import os

import scanpy
import numpy as np
import scipy.sparse as sp

import torch
from torch import optim
from torch.utils.data import DataLoader

import image.loadImage as loadImage
import gae.gae.optimizer as optimizer
import gae.gae.model
import gae.gae.preprocessing as preprocessing
import image.modelsCNN as modelsCNN

import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import scale
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import pairwise_distances

In [3]:
trainClusters=[0,1,2,3,4,5,6,7,8]
os.environ["CUDA_VISIBLE_DEVICES"] = "2" 
use_cuda=True

In [4]:
#load pretrained GAE
testepoch=9420

maskedgeName='knn20_connectivity'
hidden1=6000 #Number of units in hidden layer 1
hidden2=6000 #Number of units in hidden layer 2
# hidden3=2048
# hidden4=2048
# hidden5=128
fc_dim1=6000
# fc_dim2=128
# fc_dim3=128
# fc_dim4=128
# gcn_dim1=2600
# clf_hidden=256
adv_hidden=128

dropout=0.01
testNodes=0.1 #fraction of total nodes for testing
valNodes=0.05 #fraction of total nodes for validation
XreconWeight=20
# clfweight=20
advWeight=2
# randFeatureSubset=None
model_str='gcn_vae_xa_e2_d1_dca'
protein=None #'scaled_binary'
proteinWeights=0.001
adj_decodeName=None #gala or None
ridgeL=0.01
shareGenePi=True

num_features=2112
# training_samples=['disease13','control13','disease8','control8']
training_samples=['disease13','control13','disease8']
training_sample_X='logminmax'
standardizeX=False
# name='allk20XA_01_dca_noD8' ###rename and retrain D8 C8
name='allnoC8k20XA_02_dca_over'
modelsavepath='/mnt/external_ssd/xinyi/models/train_gae_starmap/'+name
datadir='/home/xinyiz/2021-01-13-mAD-test-dataset'

#Load data
sampleidx={'disease13':'AD_mouse9494','control13':'AD_mouse9498','disease8':'AD_mouse9723','control8':'AD_mouse9735'}
savedir=os.path.join('/home/xinyiz/starmap')
adj_dir=os.path.join(savedir,'a')

featureslist={}
if training_sample_X in ['corrected','scaled']:
    scaleddata=scanpy.read_h5ad(datadir+'/2020-12-27-starmap-mAD-scaled.h5ad')
    
    for s in sampleidx.keys():
        featureslist[s+'X_'+'corrected']=torch.tensor(scaleddata.layers['corrected'][scaleddata.obs['sample']==sampleidx[s]])
        featureslist[s+'X_'+'scaled']=torch.tensor(scaleddata.layers['scaled'][scaleddata.obs['sample']==sampleidx[s]])

else:
    scaleddata=scanpy.read_h5ad(datadir+'/2020-12-27-starmap-mAD-raw.h5ad')
    
    for s in sampleidx.keys():
        scaleddata_train=scaleddata.X[scaleddata.obs['sample']==sampleidx[s]]

        if training_sample_X=='logminmax':
            featurelog_train=np.log2(scaleddata_train+1/2)
            scaler = MinMaxScaler()
            featurelog_train_minmax=np.transpose(scaler.fit_transform(np.transpose(featurelog_train)))
            featureslist[s+'X_'+training_sample_X]=torch.tensor(featurelog_train_minmax)
        elif training_sample_X=='logminmax10':
            featurelog_train=np.log2(scaleddata_train+1/2)
            scaler = MinMaxScaler(feature_range=(0,10))
            featurelog_train_minmax=np.transpose(scaler.fit_transform(np.transpose(featurelog_train)))
            featureslist[s+'X_'+training_sample_X]=torch.tensor(featurelog_train_minmax)

adj_list={}
adj_list['disease13']=sp.load_npz(os.path.join(adj_dir,maskedgeName+'_AD_mouse9494.npz'))
adj_list['control13']=sp.load_npz(os.path.join(adj_dir,maskedgeName+'_AD_mouse9498.npz'))
adj_list['disease8']=sp.load_npz(os.path.join(adj_dir,maskedgeName+'_AD_mouse9723.npz'))
adj_list['control8']=sp.load_npz(os.path.join(adj_dir,maskedgeName+'_AD_mouse9735.npz'))

adjnormlist={}
pos_weightlist={}
normlist={}
for ai in adj_list.keys():
    adjnormlist[ai]=preprocessing.preprocess_graph(adj_list[ai])
    
    pos_weightlist[ai] = torch.tensor(float(adj_list[ai].shape[0] * adj_list[ai].shape[0] - adj_list[ai].sum()) / adj_list[ai].sum()) #using full unmasked adj
    normlist[ai] = adj_list[ai].shape[0] * adj_list[ai].shape[0] / float((adj_list[ai].shape[0] * adj_list[ai].shape[0] - adj_list[ai].sum()) * 2)
    
    adj_label=adj_list[ai] + sp.eye(adj_list[ai].shape[0])
    adj_list[ai]=torch.tensor(adj_label.todense())
                
            
if 'dca' in model_str:
    rawdata=scanpy.read_h5ad(datadir+'/2020-12-27-starmap-mAD-raw.h5ad')
    features_raw_list={}
    for s in sampleidx.keys():
#         features_raw_list[s+'X_'+'raw']=torch.tensor(rawdata.X[rawdata.obs['sample']==sampleidx[s]]).cuda()
        features_raw_list[s+'X_'+'raw']=torch.tensor(rawdata.X[rawdata.obs['sample']==sampleidx[s]])

if model_str=='gcn_vae_xa_e2_d1_dca':
    modelgae = gae.gae.model.GCNModelVAE_XA_e2_d1_DCA(num_features, hidden1,hidden2,fc_dim1, dropout)
    loss_kl=optimizer.optimizer_kl
    loss_x=optimizer.optimizer_zinb
    loss_a=optimizer.optimizer_CE
modelgae.load_state_dict(torch.load(os.path.join(modelsavepath,str(testepoch)+'.pt')))

if use_cuda:
    modelgae.cuda()

gaeLatent={}
modelgae.eval()
for s in sampleidx.keys():
    print(s)
    
    adj_norm=adjnormlist[s].cuda().float()
#     adj_label=adj_list[s].cuda().float()
    features=featureslist[s+'X_'+training_sample_X].cuda().float()
    
    if 'dca' in model_str:
        features_raw=features_raw_list[s+'X_raw'].cuda()
    num_nodes,num_features = features.shape

    if adj_decodeName==None:
        _,_,_,z, _ = modelgae(features, adj_norm)
    else:
        adj_decode=adj_decode.cuda()
        adj_recon,mu,logvar,z, features_recon = modelgae(features, adj_norm,adj_decode)
    gaeLatent[s]=z.detach()
    
gaeClusterPath='/mnt/external_ssd/xinyi/plots/train_gae_starmap/allk20XA_02_dca_over/combinedlogminmax_beforeAct/cluster/leiden_nn10mdist025n_pcs40res0.1epoch9990'
with open(gaeClusterPath, 'rb') as input:
    gaeclusterlabels = pickle.load(input)
gaeclusterlabels=gaeclusterlabels.astype(int)
    
sampleNames=None
for s in sampleidx.keys():
    sampleidx_s=sampleidx[s] 
    if sampleNames is None:
        sampleNames=np.repeat(s,np.sum(scaleddata.obs['sample']==sampleidx_s))
    else:
        sampleNames=np.concatenate((sampleNames,np.repeat(s,np.sum(scaleddata.obs['sample']==sampleidx_s))),axis=None)

cellCoord={}
for s in sampleidx.keys():
    print(s)
#     print(gaeclusterlabels[sampleNames==s][100:105])
    sampleidx_s=sampleidx[s] 
    cidx=np.repeat(False,np.sum(scaleddata.obs['sample']==sampleidx_s))
    for c in trainClusters:
        cidx=np.logical_or(gaeclusterlabels[sampleNames==s]==c,cidx)
    cellCoord[s]=(((scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx_s,['y','x']].to_numpy())/0.3).astype(int))[cidx]
    gaeLatent[s]=gaeLatent[s][cidx]
    
rawdata=None
scaleddata=None




0
0
0
0
disease13
control13
disease8
control8
disease13
control13
disease8
control8


In [5]:
# Settings
useSavedMaskedEdges=True
fastmode=False #Validate during training pass
seed=3
epochs=1200
saveFreq=4
lr=0.00001 #initial learning rate
lr_adv=0.001
weight_decay=0 #Weight for L2 loss on embedding matrix.

# batchsize=4
batchsize=500
kernel_size=4
stride=2
padding=1

# fc_dim1=6000
hidden1=64 #Number of channels in hidden layer 1
hidden2=128 
hidden3=256
hidden4=256
hidden5=96
fc_dim1=96*5*5
fc_dim2=6000
# fc_dim3=128
# fc_dim4=128
# gcn_dim1=2600
# adv_hidden=128

dropout=0.01
testNodes=0.1 #fraction of total nodes for testing
valNodes=0.05 #fraction of total nodes for validation
# clfweight=20
advWeight=2
# randFeatureSubset=None
kl_weight=0.0000001
match_weight=1000
model_str='cnn_vae'
adv=None  #'clf_fc1_eq'  #'clf_fc1_control_eq' #'clf_fc1_control'  #'clf_fc1'

pretrainedAE=None #{'name':'controlphy5XAbin_01_dca','epoch':9990}
# training_samples=['disease13','control13','disease8','control8']
training_samples=['disease13','control13','disease8']
targetBatch=None
switchFreq=4
diamThresh_mul=160
minThresh_mul=12
name='allnoC8_thresh25_02_mse1000_sm5x'
logsavepath='/mnt/external_ssd/xinyi/log/train_jointGAEcnn_starmap/'+name
modelsavepath='/mnt/external_ssd/xinyi/models/train_jointGAEcnn_starmap/'+name
plotsavepath='/mnt/external_ssd/xinyi/plots/train_jointGAEcnn_starmap/'+name

#Load data
sampleidx={'disease13':'AD_mouse9494','control13':'AD_mouse9498','disease8':'AD_mouse9723','control8':'AD_mouse9735'}
datadir='/home/xinyiz/2021-01-13-mAD-test-dataset'


if adv:
    if 'control_eq' in adv:
        sampleLabellist_ae={}
        sampleLabellist_ae['control13']=torch.tensor([0.5,0.5]).expand(adjnormlist['control13'].shape[0],-1)
        sampleLabellist_ae['control8']=torch.tensor([0.5,0.5]).expand(adjnormlist['control8'].shape[0],-1)

        sampleLabellist_d={}
        sampleLabellist_d['control13']=torch.tensor([1,0]).expand(adjnormlist['control13'].shape[0],-1)
        sampleLabellist_d['control8']=torch.tensor([0,1]).expand(adjnormlist['control8'].shape[0],-1)
    elif 'control' in adv:
        sampleLabellist_ae={}
        sampleLabellist_ae['control13']=torch.tensor([0,1]).expand(adjnormlist['control13'].shape[0],-1)
        sampleLabellist_ae['control8']=torch.tensor([1,0]).expand(adjnormlist['control8'].shape[0],-1)

        sampleLabellist_d={}
        sampleLabellist_d['control13']=torch.tensor([1,0]).expand(adjnormlist['control13'].shape[0],-1)
        sampleLabellist_d['control8']=torch.tensor([0,1]).expand(adjnormlist['control8'].shape[0],-1)        
    elif 'eq' in adv:
        sampleLabellist_ae={}
        sampleLabellist_ae['disease13']=torch.tensor([0.5,0.5,0.5,0.5]).expand(adjnormlist['disease13'].shape[0],-1)
        sampleLabellist_ae['control13']=torch.tensor([0.5,0.5,0.5,0.5]).expand(adjnormlist['control13'].shape[0],-1)
        sampleLabellist_ae['disease8']=torch.tensor([0.5,0.5,0.5,0.5]).expand(adjnormlist['disease8'].shape[0],-1)
        sampleLabellist_ae['control8']=torch.tensor([0.5,0.5,0.5,0.5]).expand(adjnormlist['control8'].shape[0],-1)

        sampleLabellist_d={}
        sampleLabellist_d['disease13']=torch.tensor([1,0,0,0]).expand(adjnormlist['disease13'].shape[0],-1)
        sampleLabellist_d['control13']=torch.tensor([0,1,0,0]).expand(adjnormlist['control13'].shape[0],-1)
        sampleLabellist_d['disease8']=torch.tensor([0,0,1,0]).expand(adjnormlist['disease8'].shape[0],-1)
        sampleLabellist_d['control8']=torch.tensor([0,0,0,1]).expand(adjnormlist['control8'].shape[0],-1)
    else:
        sampleLabellist_ae={}
        sampleLabellist_ae['disease13']=torch.tensor([0,1,1,1]).expand(adjnormlist['disease13'].shape[0],-1)
        sampleLabellist_ae['control13']=torch.tensor([1,0,1,1]).expand(adjnormlist['control13'].shape[0],-1)
        sampleLabellist_ae['disease8']=torch.tensor([1,1,0,1]).expand(adjnormlist['disease8'].shape[0],-1)
        sampleLabellist_ae['control8']=torch.tensor([1,1,1,0]).expand(adjnormlist['control8'].shape[0],-1)

        sampleLabellist_d={}
        sampleLabellist_d['disease13']=torch.tensor([1,0,0,0]).expand(adjnormlist['disease13'].shape[0],-1)
        sampleLabellist_d['control13']=torch.tensor([0,1,0,0]).expand(adjnormlist['control13'].shape[0],-1)
        sampleLabellist_d['disease8']=torch.tensor([0,0,1,0]).expand(adjnormlist['disease8'].shape[0],-1)
        sampleLabellist_d['control8']=torch.tensor([0,0,0,1]).expand(adjnormlist['control8'].shape[0],-1)
        
# Set cuda and seed
np.random.seed(seed)
if use_cuda and (not torch.cuda.is_available()):
    print('cuda not available')
    use_cuda=False
torch.manual_seed(seed)
if use_cuda:
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.enabled = True

# Load data
# if randFeatureSubset != None:
#     idx=np.random.choice(features.shape[1],randFeatureSubset,replace=False)
#     features=features[:,idx]

In [6]:
if not os.path.exists(logsavepath):
    os.mkdir(logsavepath)
if not os.path.exists(modelsavepath):
    os.mkdir(modelsavepath)
if not os.path.exists(plotsavepath):
    os.mkdir(plotsavepath)

In [7]:
# loss_match=torch.nn.MSELoss(reduction='sum')
loss_match=torch.nn.MSELoss()
# Create model
if model_str=='cnn_vae':
    modelcnn = modelsCNN.CNN_VAE(kernel_size, stride, padding, 1, hidden1, hidden2, hidden3, hidden4, hidden5, fc_dim1,fc_dim2)
    loss_kl=optimizer.optimizer_kl
    loss_x=torch.nn.MSELoss()
if model_str=='cnn_vae_sharded':
    modelcnn = modelsCNN.CNN_VAE_sharded(kernel_size, stride, padding, 1, hidden1, hidden2, hidden3, hidden4, hidden5, fc_dim1,fc_dim2)
    loss_kl=optimizer.optimizer_kl
    loss_x=torch.nn.MSELoss()
    use_cuda=False

if model_str=='cnn_vae_alexnet':
    modelcnn = modelsCNN.CNN_VAE_alexnet(fc_dim1)
    loss_kl=optimizer.optimizer_kl
    loss_x=torch.nn.MSELoss(reduction='sum')

    
if adv=='clf_fc1' or adv=='clf_fc1_eq' or adv=='clf_fc1_control' or adv=='clf_fc1_control_eq':
    modelAdv=gae.gae.model.Clf_fc1(hidden2, dropout,adv_hidden,sampleLabellist_ae['control13'].size()[1])
    loss_adv=optimizer.optimizer_CEclf
    
if adv=='clf_linear1' or adv=='clf_linear1_control':
    modelAdv=gae.gae.model.Clf_linear1(hidden2, dropout,sampleLabellist_ae['control13'].size()[1])
    loss_adv=optimizer.optimizer_CEclf
        


In [8]:
features=None
features_raw=None
adj_norm=None
_=None
modelgae.cpu()
torch.cuda.empty_cache()


In [9]:
#train cnn only, matching gae

# model.load_state_dict(torch.load(os.path.join(modelsavepath,str(9360)+'.pt')))
# epochs=20000
# if pretrainedAE:
#     print('loading '+pretrainedAE['name']+' epoch '+str(pretrainedAE['epoch']))
#     model.load_state_dict(torch.load(os.path.join('/mnt/xinyi/pamrats/models/train_gae_starmap/'+pretrainedAE['name'],str(pretrainedAE['epoch'])+'.pt')))

if use_cuda:
    modelcnn.cuda()
    if adv:
        modelAdv.cuda()
# modelcnn.load_state_dict(torch.load(os.path.join(modelsavepath,'370.pt')))

optimizerVAEXA = optim.Adam(modelcnn.parameters(), lr=lr, weight_decay=weight_decay)
if adv:
    optimizerAdv=optim.Adam(modelAdv.parameters(), lr=lr_adv, weight_decay=weight_decay)    


def train(epoch):
    t = time.time()
    modelcnn.train()
    
    loss_kl_train_all=0
    loss_x_train_all=0
    loss_match_train_all=0
    loss_all=0
    ntrainBatches=int(np.ceil(train_nodes_idx.shape[0]/batchsize))
    for i in range(ntrainBatches):
#         if i%200==0:
#             print(i)
        trainIdx=train_nodes_idx[i*batchsize:min((i+1)*batchsize,train_nodes_idx.shape[0])]
        trainInput=torch.tensor(trainInputnp[trainIdx.numpy()])
#         print(trainInput.shape)

        if torch.sum(trainInput)==0:
            print('all zeros '+str(i))

        if use_cuda:
            trainInput=trainInput.cuda().float()
        optimizerVAEXA.zero_grad()

        recon, z, mu, logvar = modelcnn(trainInput)

        if adv and (training_samples_t in list(sampleLabellist_ae.keys())) and (training_samples_t != targetBatch):
            modelAdv.eval()
            advOut=modelAdv(z)

        loss_kl_train=loss_kl(mu, logvar,reduction='sum')
        loss_x_train=loss_x(recon, trainInput)
        loss_match_train=loss_match(z,gaetarget[trainIdx])
        loss=loss_kl_train*kl_weight+loss_x_train+loss_match_train*match_weight
        
        
        loss_kl_train_all+=loss_kl_train.item()
        loss_x_train_all+=loss_x_train.item()
        loss_match_train_all+=loss_match_train.item()
        loss_all+=loss.item()
        
        if adv and (training_samples_t in list(sampleLabellist_ae.keys())) and (training_samples_t != targetBatch):
            loss_adv_train=loss_adv(advOut,sampleLabel_ae,train_nodes_idx)
            loss+=loss_adv_train*advWeight
            
        loss.backward()
        optimizerVAEXA.step()

    loss_kl_train_all=loss_kl_train_all/train_nodes_idx.shape[0]
    loss_x_train_all=loss_x_train_all/train_nodes_idx.shape[0]
    loss_match_train_all=loss_match_train_all/train_nodes_idx.shape[0]
    loss_all=loss_all/train_nodes_idx.shape[0]
    
    with torch.no_grad():
        modelcnn.eval()
        loss_x_val_all=0
        loss_match_val_all=0
        loss_val_all=0
        nvalBatches=int(np.ceil(val_nodes_idx.shape[0]/batchsize))
        for i in range(nvalBatches):
            valIdx=val_nodes_idx[i*batchsize:min((i+1)*batchsize,val_nodes_idx.shape[0])]
            valInput=torch.tensor(trainInputnp[valIdx.numpy()])
            if use_cuda:
                valInput=valInput.cuda().float()
            recon,z, mu, logvar = modelcnn(valInput)

            if adv and (training_samples_t in list(sampleLabellist_ae.keys())) and (training_samples_t != targetBatch):
                advOut=modelAdv(z)

            loss_x_val=loss_x(recon, valInput).item()
            loss_match_val=loss_match(z,gaetarget[valIdx]).item()

            loss_x_val_all+=loss_x_val
            loss_match_val_all+=loss_match_val
            loss_val_all+=(loss_x_val+loss_match_val*match_weight)

            if adv and (training_samples_t in list(sampleLabellist_ae.keys())) and (training_samples_t != targetBatch):
                loss_adv_val=loss_adv(advOut,sampleLabel_ae,val_nodes_idx)
                loss_val+=loss_adv_val*advWeight
        loss_x_val_all=loss_x_val_all/val_nodes_idx.shape[0]
        loss_match_val_all=loss_match_val_all/val_nodes_idx.shape[0]
        loss_val_all=loss_val_all/val_nodes_idx.shape[0]
    
    print(training_samples_t+' Epoch: {:04d}'.format(epoch),
          'loss_train: {:.4f}'.format(loss_all),
          'loss_kl_train: {:.4f}'.format(loss_kl_train_all),
          'loss_x_train: {:.4f}'.format(loss_x_train_all),
          'loss_match_train: {:.4f}'.format(loss_match_train_all),
          'loss_val: {:.4f}'.format(loss_val_all),
          'loss_x_val: {:.4f}'.format(loss_x_val_all),
          'loss_match_val: {:.4f}'.format(loss_match_val_all),
          'time: {:.4f}s'.format(time.time() - t))
    if adv and (training_samples_t in list(sampleLabellist_ae.keys())) and (training_samples_t != targetBatch):
        print('loss_adv_train: {:.4f}'.format(loss_adv_train.item()),
              'loss_adv_val: {:.4f}'.format(loss_adv_val.item())
             )
#     return loss.item(),loss_x_train.item(),loss_val.item(),loss_x_val.item()
#     return loss.item(),loss_kl_train.item(),loss_x_train.item(),loss_val.item(),loss_x_val.item()
    if adv:
        if (training_samples_t in list(sampleLabellist_ae.keys())) and (training_samples_t != targetBatch):
            return float(loss),float(loss_kl_train),float(loss_x_train),float(loss_a_train),float(loss_val),float(loss_x_val),float(loss_a_val),float(loss_adv_train),float(loss_adv_val)        
        else:
            return float(loss),float(loss_kl_train),float(loss_x_train),float(loss_a_train),float(loss_val),float(loss_x_val),float(loss_a_val),None,None   
    else:
        return loss_all,loss_kl_train_all,loss_x_train_all,loss_match_train_all,loss_val_all,loss_x_val_all,loss_match_val_all

def train_discriminator(epoch):
    t = time.time()
    model.eval()
    
    if adj_decodeName==None:
        adj_recon,mu,logvar,z,features_recon = model(features, adj_norm)
    #     features_recon, z, mu, logvar=model(features.float())
    else:
        adj_recon,mu,logvar,z,features_recon = model(features, adj_norm,adj_decode)
        
    
    if clf:
        modelClf.eval()
        clfOut=modelClf(z)
        
    modelAdv.train()
    optimizerAdv.zero_grad()
    advOut=modelAdv(z)
    
    loss_adv_train=loss_adv(advOut,sampleLabel_d,train_nodes_idx)
    loss = loss_adv_train*advWeight
    
    loss.backward()
    optimizerAdv.step()

    modelAdv.eval()
    advOut=modelAdv(z)
    loss_adv_val=loss_adv(advOut,sampleLabel_d,val_nodes_idx)
    loss_val=loss_adv_val*advWeight
    print(training_samples_t+' Epoch: {:04d}'.format(epoch),
          'loss_adv_train: {:.4f}'.format(loss_adv_train.item()),
          'loss_adv_val: {:.4f}'.format(loss_adv_val.item()),
          'time: {:.4f}s'.format(time.time() - t))
    return float(loss_adv_train),float(loss_adv_val)
    
# print('cross-validation ',seti)
train_loss_ep=[None]*epochs
train_loss_kl_ep=[None]*epochs
train_loss_x_ep=[None]*epochs
train_loss_match_ep=[None]*epochs
# train_loss_adv_ep=[None]*epochs
# train_loss_advD_ep=[None]*epochs
val_loss_ep=[None]*epochs
val_loss_x_ep=[None]*epochs
val_loss_match_ep=[None]*epochs
# val_loss_adv_ep=[None]*epochs
# val_loss_advD_ep=[None]*epochs
t_ep=time.time()


# pre_t=4
pre_t=-1
for ep in range(epochs):
# for ep in range(10000,20000):
    t=int(ep/switchFreq)%len(training_samples)
    if t!=pre_t:
        pre_t=t
        trainInputnp=None
        torch.cuda.empty_cache()
        training_samples_t=training_samples[t]
        trainInputnp=loadImage.load_cellCentroid(cellCoord[training_samples_t],sampleidx[training_samples_t],datadir,diamThresh_mul,ifFlip=False,seed=3,imagename='pi_sum.tif',minmaxscale=True,nchannels=1)
        gaetarget=gaeLatent[training_samples_t]
    
#         maskpath=os.path.join(savedir,'trainMask',training_samples_t+'_'+maskedgeName+'_seed'+str(seed)+'.pkl')
#         if useSavedMaskedEdges and os.path.exists(maskpath):
#     #         print('opening saved')
#             with open(maskpath, 'rb') as input:
#                 maskedgeres = pickle.load(input)
#         else:
            # construct training, validation, and test sets
        maskedgeres= preprocessing.mask_nodes_edges(trainInputnp.shape[0],testNodeSize=testNodes,valNodeSize=valNodes,seed=seed)
#             with open(maskpath, 'wb') as output:
#                 pickle.dump(maskedgeres, output, pickle.HIGHEST_PROTOCOL)
        train_nodes_idx,val_nodes_idx,test_nodes_idx = maskedgeres
#         if use_cuda:
#             train_nodes_idx=train_nodes_idx.cuda()
#             val_nodes_idx=val_nodes_idx.cuda()
#             test_nodes_idx=test_nodes_idx.cuda()
    
    if adv and (training_samples_t in list(sampleLabellist_ae.keys())):
#         sampleLabel_ae=sampleLabellist_ae[training_samples_t]
#         sampleLabel_d=sampleLabellist_d[training_samples_t]
        sampleLabel_ae=sampleLabellist_ae[training_samples_t].cuda().float()
        sampleLabel_d=sampleLabellist_d[training_samples_t].cuda().float()
    
    if adv:
        train_loss_ep[ep],train_loss_kl_ep[ep],train_loss_x_ep[ep],train_loss_a_ep[ep],val_loss_ep[ep],val_loss_x_ep[ep],val_loss_a_ep[ep],train_loss_adv_ep[ep],val_loss_adv_ep[ep]=train(ep)
        if (training_samples_t in list(sampleLabellist_ae.keys())):
            train_loss_advD_ep[ep],val_loss_advD_ep[ep]=train_discriminator(ep)
    else:
        train_loss_ep[ep],train_loss_kl_ep[ep],train_loss_x_ep[ep],train_loss_match_ep[ep],val_loss_ep[ep],val_loss_x_ep[ep],val_loss_match_ep[ep]=train(ep)

        
    if ep%saveFreq == 0 and ep != 0:
        torch.save(modelcnn.cpu().state_dict(), os.path.join(modelsavepath,str(ep)+'.pt'))
    if use_cuda:
        modelcnn.cuda()
        torch.cuda.empty_cache()
print(' total time: {:.4f}s'.format(time.time() - t_ep))


(22210, 22344)
disease13 Epoch: 0000 loss_train: 2.7386 loss_kl_train: 1349.2562 loss_x_train: 0.0004 loss_match_train: 0.0027 loss_val: 0.0658 loss_x_val: 0.0005 loss_match_val: 0.0001 time: 8.7569s
disease13 Epoch: 0001 loss_train: 1.8810 loss_kl_train: 1026.6840 loss_x_train: 0.0004 loss_match_train: 0.0019 loss_val: 0.1295 loss_x_val: 0.0003 loss_match_val: 0.0001 time: 8.7696s
disease13 Epoch: 0002 loss_train: 1.3901 loss_kl_train: 1176.7770 loss_x_train: 0.0003 loss_match_train: 0.0014 loss_val: 0.1631 loss_x_val: 0.0003 loss_match_val: 0.0002 time: 8.7727s
disease13 Epoch: 0003 loss_train: 1.0658 loss_kl_train: 1528.6834 loss_x_train: 0.0003 loss_match_train: 0.0011 loss_val: 0.1683 loss_x_val: 0.0002 loss_match_val: 0.0002 time: 8.8009s
(22355, 18953)
control13 Epoch: 0004 loss_train: 0.8925 loss_kl_train: 1929.5942 loss_x_train: 0.0002 loss_match_train: 0.0009 loss_val: 0.0884 loss_x_val: 0.0001 loss_match_val: 0.0001 time: 6.7999s
control13 Epoch: 0005 loss_train: 0.7215 loss

(22294, 19552)
disease8 Epoch: 0044 loss_train: 0.0239 loss_kl_train: 12041.8411 loss_x_train: 0.0000 loss_match_train: 0.0000 loss_val: 0.0092 loss_x_val: 0.0001 loss_match_val: 0.0000 time: 6.5543s
disease8 Epoch: 0045 loss_train: 0.0233 loss_kl_train: 12140.1825 loss_x_train: 0.0000 loss_match_train: 0.0000 loss_val: 0.0090 loss_x_val: 0.0001 loss_match_val: 0.0000 time: 6.5488s
disease8 Epoch: 0046 loss_train: 0.0228 loss_kl_train: 12234.6666 loss_x_train: 0.0000 loss_match_train: 0.0000 loss_val: 0.0089 loss_x_val: 0.0001 loss_match_val: 0.0000 time: 6.5609s
disease8 Epoch: 0047 loss_train: 0.0223 loss_kl_train: 12327.1504 loss_x_train: 0.0000 loss_match_train: 0.0000 loss_val: 0.0089 loss_x_val: 0.0001 loss_match_val: 0.0000 time: 6.5505s
(22210, 22344)
disease13 Epoch: 0048 loss_train: 0.0206 loss_kl_train: 12422.3309 loss_x_train: 0.0000 loss_match_train: 0.0000 loss_val: 0.0113 loss_x_val: 0.0002 loss_match_val: 0.0000 time: 8.9789s
disease13 Epoch: 0049 loss_train: 0.0200 los

(22355, 18953)
control13 Epoch: 0088 loss_train: 0.0120 loss_kl_train: 15566.3732 loss_x_train: 0.0000 loss_match_train: 0.0000 loss_val: 0.0065 loss_x_val: 0.0001 loss_match_val: 0.0000 time: 6.9814s
control13 Epoch: 0089 loss_train: 0.0118 loss_kl_train: 15621.6943 loss_x_train: 0.0000 loss_match_train: 0.0000 loss_val: 0.0064 loss_x_val: 0.0001 loss_match_val: 0.0000 time: 7.0003s
control13 Epoch: 0090 loss_train: 0.0117 loss_kl_train: 15676.6599 loss_x_train: 0.0000 loss_match_train: 0.0000 loss_val: 0.0064 loss_x_val: 0.0001 loss_match_val: 0.0000 time: 7.0357s
control13 Epoch: 0091 loss_train: 0.0116 loss_kl_train: 15730.8218 loss_x_train: 0.0000 loss_match_train: 0.0000 loss_val: 0.0064 loss_x_val: 0.0001 loss_match_val: 0.0000 time: 7.0331s
(22294, 19552)
disease8 Epoch: 0092 loss_train: 0.0114 loss_kl_train: 15790.9455 loss_x_train: 0.0000 loss_match_train: 0.0000 loss_val: 0.0073 loss_x_val: 0.0001 loss_match_val: 0.0000 time: 6.5180s
disease8 Epoch: 0093 loss_train: 0.0113 l

(22210, 22344)
disease13 Epoch: 0132 loss_train: 0.0076 loss_kl_train: 17714.3633 loss_x_train: 0.0000 loss_match_train: 0.0000 loss_val: 0.0073 loss_x_val: 0.0001 loss_match_val: 0.0000 time: 9.0295s
disease13 Epoch: 0133 loss_train: 0.0076 loss_kl_train: 17764.6331 loss_x_train: 0.0000 loss_match_train: 0.0000 loss_val: 0.0073 loss_x_val: 0.0001 loss_match_val: 0.0000 time: 9.0187s
disease13 Epoch: 0134 loss_train: 0.0075 loss_kl_train: 17814.2927 loss_x_train: 0.0000 loss_match_train: 0.0000 loss_val: 0.0073 loss_x_val: 0.0001 loss_match_val: 0.0000 time: 9.0210s
disease13 Epoch: 0135 loss_train: 0.0075 loss_kl_train: 17863.4662 loss_x_train: 0.0000 loss_match_train: 0.0000 loss_val: 0.0073 loss_x_val: 0.0001 loss_match_val: 0.0000 time: 9.0145s
(22355, 18953)
control13 Epoch: 0136 loss_train: 0.0090 loss_kl_train: 17903.9925 loss_x_train: 0.0000 loss_match_train: 0.0000 loss_val: 0.0058 loss_x_val: 0.0000 loss_match_val: 0.0000 time: 6.9458s
control13 Epoch: 0137 loss_train: 0.0089

(22294, 19552)
disease8 Epoch: 0176 loss_train: 0.0080 loss_kl_train: 19324.1070 loss_x_train: 0.0000 loss_match_train: 0.0000 loss_val: 0.0065 loss_x_val: 0.0000 loss_match_val: 0.0000 time: 6.5083s
disease8 Epoch: 0177 loss_train: 0.0080 loss_kl_train: 19352.9541 loss_x_train: 0.0000 loss_match_train: 0.0000 loss_val: 0.0065 loss_x_val: 0.0000 loss_match_val: 0.0000 time: 6.5464s
disease8 Epoch: 0178 loss_train: 0.0080 loss_kl_train: 19380.2371 loss_x_train: 0.0000 loss_match_train: 0.0000 loss_val: 0.0065 loss_x_val: 0.0000 loss_match_val: 0.0000 time: 6.5410s
disease8 Epoch: 0179 loss_train: 0.0080 loss_kl_train: 19407.2612 loss_x_train: 0.0000 loss_match_train: 0.0000 loss_val: 0.0065 loss_x_val: 0.0000 loss_match_val: 0.0000 time: 6.5348s
(22210, 22344)
disease13 Epoch: 0180 loss_train: 0.0065 loss_kl_train: 19433.9431 loss_x_train: 0.0000 loss_match_train: 0.0000 loss_val: 0.0067 loss_x_val: 0.0000 loss_match_val: 0.0000 time: 8.9769s
disease13 Epoch: 0181 loss_train: 0.0065 los

...

(22355, 18953)
control13 Epoch: 1012 loss_train: 0.0068 loss_kl_train: 23628.1150 loss_x_train: 0.0000 loss_match_train: 0.0000 loss_val: 0.0048 loss_x_val: 0.0000 loss_match_val: 0.0000 time: 6.9885s
control13 Epoch: 1013 loss_train: 0.0068 loss_kl_train: 23640.2124 loss_x_train: 0.0000 loss_match_train: 0.0000 loss_val: 0.0048 loss_x_val: 0.0000 loss_match_val: 0.0000 time: 7.0065s
control13 Epoch: 1014 loss_train: 0.0068 loss_kl_train: 23638.6448 loss_x_train: 0.0000 loss_match_train: 0.0000 loss_val: 0.0048 loss_x_val: 0.0000 loss_match_val: 0.0000 time: 7.0198s
control13 Epoch: 1015 loss_train: 0.0068 loss_kl_train: 23635.6481 loss_x_train: 0.0000 loss_match_train: 0.0000 loss_val: 0.0048 loss_x_val: 0.0000 loss_match_val: 0.0000 time: 7.1471s
(22294, 19552)
disease8 Epoch: 1016 loss_train: 0.0069 loss_kl_train: 23640.8072 loss_x_train: 0.0000 loss_match_train: 0.0000 loss_val: 0.0059 loss_x_val: 0.0000 loss_match_val: 0.0000 time: 6.7041s
disease8 Epoch: 1017 loss_train: 0.0069 l

(22210, 22344)
disease13 Epoch: 1056 loss_train: 0.0054 loss_kl_train: 23603.2715 loss_x_train: 0.0000 loss_match_train: 0.0000 loss_val: 0.0059 loss_x_val: 0.0000 loss_match_val: 0.0000 time: 9.0042s
disease13 Epoch: 1057 loss_train: 0.0054 loss_kl_train: 23607.6270 loss_x_train: 0.0000 loss_match_train: 0.0000 loss_val: 0.0058 loss_x_val: 0.0000 loss_match_val: 0.0000 time: 9.0159s
disease13 Epoch: 1058 loss_train: 0.0054 loss_kl_train: 23622.1012 loss_x_train: 0.0000 loss_match_train: 0.0000 loss_val: 0.0058 loss_x_val: 0.0000 loss_match_val: 0.0000 time: 9.0181s
disease13 Epoch: 1059 loss_train: 0.0054 loss_kl_train: 23624.2760 loss_x_train: 0.0000 loss_match_train: 0.0000 loss_val: 0.0058 loss_x_val: 0.0000 loss_match_val: 0.0000 time: 9.0327s
(22355, 18953)
control13 Epoch: 1060 loss_train: 0.0068 loss_kl_train: 23627.4545 loss_x_train: 0.0000 loss_match_train: 0.0000 loss_val: 0.0048 loss_x_val: 0.0000 loss_match_val: 0.0000 time: 6.9766s
control13 Epoch: 1061 loss_train: 0.0068

disease8 Epoch: 1100 loss_train: 0.0069 loss_kl_train: 23643.4653 loss_x_train: 0.0000 loss_match_train: 0.0000 loss_val: 0.0059 loss_x_val: 0.0000 loss_match_val: 0.0000 time: 6.4938s
disease8 Epoch: 1101 loss_train: 0.0069 loss_kl_train: 23634.1707 loss_x_train: 0.0000 loss_match_train: 0.0000 loss_val: 0.0059 loss_x_val: 0.0000 loss_match_val: 0.0000 time: 6.5343s
disease8 Epoch: 1102 loss_train: 0.0069 loss_kl_train: 23620.3228 loss_x_train: 0.0000 loss_match_train: 0.0000 loss_val: 0.0059 loss_x_val: 0.0000 loss_match_val: 0.0000 time: 6.5221s
disease8 Epoch: 1103 loss_train: 0.0069 loss_kl_train: 23616.2580 loss_x_train: 0.0000 loss_match_train: 0.0000 loss_val: 0.0059 loss_x_val: 0.0000 loss_match_val: 0.0000 time: 6.5318s
(22210, 22344)
disease13 Epoch: 1104 loss_train: 0.0054 loss_kl_train: 23602.7395 loss_x_train: 0.0000 loss_match_train: 0.0000 loss_val: 0.0058 loss_x_val: 0.0000 loss_match_val: 0.0000 time: 9.0226s
disease13 Epoch: 1105 loss_train: 0.0054 loss_kl_train: 236

(22355, 18953)
control13 Epoch: 1144 loss_train: 0.0068 loss_kl_train: 23634.2566 loss_x_train: 0.0000 loss_match_train: 0.0000 loss_val: 0.0048 loss_x_val: 0.0000 loss_match_val: 0.0000 time: 6.9929s
control13 Epoch: 1145 loss_train: 0.0068 loss_kl_train: 23649.3062 loss_x_train: 0.0000 loss_match_train: 0.0000 loss_val: 0.0048 loss_x_val: 0.0000 loss_match_val: 0.0000 time: 7.0029s
control13 Epoch: 1146 loss_train: 0.0068 loss_kl_train: 23642.2809 loss_x_train: 0.0000 loss_match_train: 0.0000 loss_val: 0.0048 loss_x_val: 0.0000 loss_match_val: 0.0000 time: 6.9912s
control13 Epoch: 1147 loss_train: 0.0068 loss_kl_train: 23635.7555 loss_x_train: 0.0000 loss_match_train: 0.0000 loss_val: 0.0048 loss_x_val: 0.0000 loss_match_val: 0.0000 time: 7.0727s
(22294, 19552)
disease8 Epoch: 1148 loss_train: 0.0069 loss_kl_train: 23642.1632 loss_x_train: 0.0000 loss_match_train: 0.0000 loss_val: 0.0059 loss_x_val: 0.0000 loss_match_val: 0.0000 time: 6.5095s
disease8 Epoch: 1149 loss_train: 0.0069 l

(22210, 22344)
disease13 Epoch: 1188 loss_train: 0.0054 loss_kl_train: 23604.7672 loss_x_train: 0.0000 loss_match_train: 0.0000 loss_val: 0.0058 loss_x_val: 0.0000 loss_match_val: 0.0000 time: 9.0047s
disease13 Epoch: 1189 loss_train: 0.0054 loss_kl_train: 23610.8030 loss_x_train: 0.0000 loss_match_train: 0.0000 loss_val: 0.0058 loss_x_val: 0.0000 loss_match_val: 0.0000 time: 9.0353s
disease13 Epoch: 1190 loss_train: 0.0054 loss_kl_train: 23627.2276 loss_x_train: 0.0000 loss_match_train: 0.0000 loss_val: 0.0058 loss_x_val: 0.0000 loss_match_val: 0.0000 time: 9.0226s
disease13 Epoch: 1191 loss_train: 0.0054 loss_kl_train: 23627.6732 loss_x_train: 0.0000 loss_match_train: 0.0000 loss_val: 0.0058 loss_x_val: 0.0000 loss_match_val: 0.0000 time: 9.0627s
(22355, 18953)
control13 Epoch: 1192 loss_train: 0.0068 loss_kl_train: 23631.1960 loss_x_train: 0.0000 loss_match_train: 0.0000 loss_val: 0.0048 loss_x_val: 0.0000 loss_match_val: 0.0000 time: 7.0012s
control13 Epoch: 1193 loss_train: 0.0068

In [11]:
with open(os.path.join(logsavepath,'train_loss'), 'wb') as output:
    pickle.dump(train_loss_ep, output, pickle.HIGHEST_PROTOCOL)
with open(os.path.join(logsavepath,'train_loss_kl'), 'wb') as output:
    pickle.dump(train_loss_kl_ep, output, pickle.HIGHEST_PROTOCOL)
with open(os.path.join(logsavepath,'train_loss_x'), 'wb') as output:
    pickle.dump(train_loss_x_ep, output, pickle.HIGHEST_PROTOCOL)
with open(os.path.join(logsavepath,'train_loss_match'), 'wb') as output:
    pickle.dump(train_loss_match_ep, output, pickle.HIGHEST_PROTOCOL)
with open(os.path.join(logsavepath,'val_loss'), 'wb') as output:
    pickle.dump(val_loss_ep, output, pickle.HIGHEST_PROTOCOL)
with open(os.path.join(logsavepath,'val_loss_x'), 'wb') as output:
    pickle.dump(val_loss_x_ep, output, pickle.HIGHEST_PROTOCOL)
with open(os.path.join(logsavepath,'val_loss_match'), 'wb') as output:
    pickle.dump(val_loss_match_ep, output, pickle.HIGHEST_PROTOCOL)
if adv:
    with open(os.path.join(logsavepath,'train_loss_adv'), 'wb') as output:
        pickle.dump(train_loss_adv_ep, output, pickle.HIGHEST_PROTOCOL)
    with open(os.path.join(logsavepath,'val_loss_adv'), 'wb') as output:
        pickle.dump(val_loss_adv_ep, output, pickle.HIGHEST_PROTOCOL)
    with open(os.path.join(logsavepath,'train_loss_advD'), 'wb') as output:
        pickle.dump(train_loss_advD_ep, output, pickle.HIGHEST_PROTOCOL)
    with open(os.path.join(logsavepath,'val_loss_advD'), 'wb') as output:
        pickle.dump(val_loss_advD_ep, output, pickle.HIGHEST_PROTOCOL)


In [ ]:
plt.plot(np.arange(epochs),train_loss_x_ep)
plt.plot(np.arange(epochs),val_loss_x_ep)
# plt.ylim((0,100))
# plt.xlim((0,3000))
plt.legend(['training x recon loss','validation x recon loss'],loc='upper right')
plt.savefig(os.path.join(plotsavepath,'loss_seed3_x.jpg'))
plt.show()


In [13]:
# name='c13k20XA_07_dca'
# logsavepath='/mnt/xinyi/pamrats/log/train_gae_starmap/'+name
# with open(os.path.join(logsavepath,'val_loss_a'), 'rb') as output:
#     val_loss_a_ep=pickle.load(output)
np.argmin(val_loss_x_ep[:1340])
# np.where(np.logical_not(np.isfinite(val_loss_ep[:])))
# print(val_loss_ep[35])

872